# Data Integration: IS 477 Final Project
Sarah Wheeler & Megan Sia

# References:

Google. (2025). Google Colaboratory. Retrieved December 7, 2025, from https://colab.research.google.com/

Mathieu, E., Ritchie, H., Ortiz-Ospina, E. et al. A global database of COVID-19 vaccinations. Natural Humanities Behavior (2024). https://doi.org/10.1038/s41562-021-01122-8

Thakur, N. (2024). Five Years of COVID-19 Discourse on Instagram: A Labeled Instagram Dataset of Over Half a Million Posts for Multilingual Sentiment Analysis [Data set]. Zenodo. https://doi.org/10.5281/zenodo.13896353

Thakur N., “Five Years of COVID-19 Discourse on Instagram: A Labeled Instagram Dataset of Over Half a Million Posts for Multilingual Sentiment Analysis”, Proceedings of the 7th International Conference on Machine Learning and Natural Language Processing (MLNLP 2024), Chengdu, China, October 18-20, 2024 (Paper accepted for publication, Preprint available at: https://arxiv.org/abs/2410.03293)

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive') #We uploaded the datasets (find in the GitHub repo) to our Google drive, and accessed them from there.

Mounted at /content/drive


In [3]:
insta_df = pd.read_excel("/content/drive/My Drive/Dataset.xlsx") # (Thakur, 2025)
vaccine_df = pd.read_csv("/content/drive/My Drive/owid-covid-data.csv") # (Mathieu et al., 2024)

In [ ]:
#Uncomment if running in an IDE other than Colab
#from pathlib import Path

#DATA_DIR = Path("data")

#insta_df = pd.read_excel(DATA_DIR / "Dataset.xlsx")
#vaccine_df = pd.read_csv(DATA_DIR / "owid-covid-data.csv")

In [4]:
insta_df.head(4)

,PostID,Post Description,Date,Language Code,Full Language,Sentiment
0,B7mbLCVhYIf,QUESTIONS AND ANSWERS ON CORONAVIRUS PT. 2\n\n...,01/21/2020,en,English,positive
1,B7o2ozppOnS,#Viral China no deja salir a #11millones de ci...,01/22/2020,es,Spanish,negative
2,B7nMy1Xl7st,Önce tedbir sonra tevekkül inşallah. Wuhanda o...,01/22/2020,tr,Turkish,negative
3,B7m7M3SgvI1,Using humor to bring attention to a serious ma...,01/22/2020,en,English,negative


In [5]:
vaccine_df.head(4)

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-01-03,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-01-04,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-01-05,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-01-06,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN


In [6]:
vaccine_df_agg = vaccine_df[["date", "new_vaccinations", "total_vaccinations", "people_vaccinated", "total_boosters"]].groupby('date').sum().reset_index()
vaccine_df_agg.tail(10)

,date,new_vaccinations,total_vaccinations,people_vaccinated,total_boosters
1439,2023-12-10,3804.0,4.267026e+10,1.749912e+10,8.819447e+09
1440,2023-12-11,18707.0,4.163527e+10,1.710047e+10,8.532902e+09
1441,2023-12-12,30806.0,4.161688e+10,1.709233e+10,8.529704e+09
1442,2023-12-13,3080.0,4.161690e+10,1.709233e+10,8.529716e+09
1443,2023-12-14,6181.0,4.163336e+10,1.709970e+10,8.532317e+09
1444,2023-12-15,4047.0,4.161678e+10,1.709224e+10,8.529759e+09
1445,2023-12-16,498.0,4.046472e+10,1.663369e+10,8.278184e+09
1446,2023-12-17,162.0,3.761262e+10,1.562510e+10,7.439375e+09
1447,2023-12-18,102.0,3.761734e+10,1.562510e+10,7.439376e+09
1448,2023-12-19,0.0,0.000000e+00,0.000000e+00,0.000000e+00


In [7]:
df = vaccine_df.copy()

num_cols = ['total_vaccinations','people_vaccinated','total_boosters','new_vaccinations']
for c in num_cols:
    if c in df:
        df[c] = pd.to_numeric(df[c], errors='coerce')

if 'location' in df:
    bad_groups = {
        'World','Africa','Asia','Europe','European Union','High income','Upper middle income',
        'Lower middle income','Low income','North America','South America','Oceania'
    }
    df = df[~df['location'].isin(bad_groups)]

timestamp_col = 'report_timestamp'
if timestamp_col in df.columns:
    df = (df.sort_values(['location','date',timestamp_col])
            .drop_duplicates(['location','date'], keep='last'))
else:
    df = (df.groupby(['location','date'], as_index=False)
            .agg({
                'total_vaccinations':'max',
                'people_vaccinated':'max',
                'total_boosters':'max',
                'new_vaccinations':'max'
            }))

for c in ['total_vaccinations','people_vaccinated','total_boosters']:
    if c in df:
        df[c] = df.groupby('location')[c].cummax()

df = df.sort_values(['location','date'])
df['new_vacc_from_total'] = (
    df.groupby('location')['total_vaccinations'].diff().clip(lower=0)
)

if 'total_boosters' in df:
    df['new_boosters_from_total'] = (
        df.groupby('location')['total_boosters'].diff().clip(lower=0)
    )

global_daily = (
    df.groupby('date', as_index=False)
      .agg(
          total_vaccinations=('total_vaccinations','sum'),
          people_vaccinated=('people_vaccinated','sum'),
          total_boosters=('total_boosters','sum'),
          new_vaccinations=('new_vacc_from_total','sum')
      )
)

global_daily.describe()

,total_vaccinations,people_vaccinated,total_boosters,new_vaccinations
count,1.449000e+03,1.449000e+03,1.449000e+03,1.449000e+03
mean,4.107612e+09,1.402494e+09,4.062543e+08,7.489611e+06
std,3.668006e+09,1.142931e+09,4.511299e+08,1.135218e+07
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.668165e+06,3.588923e+06,0.000000e+00,3.469000e+03
50%,3.292783e+09,1.400179e+09,2.854744e+08,1.298606e+06
75%,8.249514e+09,2.375760e+09,8.444057e+08,1.070549e+07
max,1.038119e+10,4.396591e+09,1.930041e+09,4.711418e+07


In [9]:
insta_df['Date'] = pd.to_datetime(insta_df['Date'], format='%m/%d/%Y')
global_daily['date'] = pd.to_datetime(global_daily['date'])
display(insta_df.info())
display(global_daily.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500153 entries, 0 to 500152
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   PostID            500153 non-null  object        
 1   Post Description  500153 non-null  object        
 2   Date              500153 non-null  datetime64[ns]
 3   Language Code     500153 non-null  object        
 4   Full Language     500153 non-null  object        
 5   Sentiment         500153 non-null  object        
dtypes: datetime64[ns](1), object(5)
memory usage: 22.9+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1449 entries, 0 to 1448
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   date                1449 non-null   datetime64[ns]
 1   total_vaccinations  1449 non-null   float64       
 2   people_vaccinated   1449 non-null   float64       
 3   total_boosters      1449 non-null   float64       
 4   new_vaccinations    1449 non-null   float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 56.7 KB


None

In [10]:
merged_df = pd.merge(insta_df, global_daily, how='left', left_on='Date', right_on='date')
merged_df = merged_df[merged_df["date"] <= "2023-12-18 00:00:00"]
merged_df.tail(10)

,PostID,Post Description,Date,Language Code,Full Language,Sentiment,date,total_vaccinations,people_vaccinated,total_boosters,new_vaccinations
363497,C0-ldZkSYmu,Hai Dandelioners 😍\n\nLahir lagi karya baru di...,2023-12-18,id,Indonesian,positive,2023-12-18,2.284120e+09,1.055558e+09,244519251.0,102.0
363498,C0-hInqScYL,🍎３３８🍎\n\n 🌼コロナワクチンを打つ人へ🌼\n\n コロナワクチンを接種す...,2023-12-18,ja,Japanese,negative,2023-12-18,2.284120e+09,1.055558e+09,244519251.0,102.0
363499,C0-m55BSRfg,Hai Dandelioners 😍\n\nLahir lagi karya baru di...,2023-12-18,id,Indonesian,neutral,2023-12-18,2.284120e+09,1.055558e+09,244519251.0,102.0
363500,C0-ThIJoma-,Insomnio.. no puedo dormir el dolor no me dej...,2023-12-18,es,Spanish,positive,2023-12-18,2.284120e+09,1.055558e+09,244519251.0,102.0
363501,C0-ZmYCvF7r,👉🏻Millions of �NSW drivers are set to get a de...,2023-12-18,en,English,negative,2023-12-18,2.284120e+09,1.055558e+09,244519251.0,102.0
363502,C0-jxWdP8H7,ทิ้งท้ายปี2023 ได้เยี่ยมจริงๆ ครั้งแรกในชีวิตเ...,2023-12-18,th,Thai,negative,2023-12-18,2.284120e+09,1.055558e+09,244519251.0,102.0
363503,C0-e4m9Ogbw,What are some of your favorite things about RT...,2023-12-18,en,English,positive,2023-12-18,2.284120e+09,1.055558e+09,244519251.0,102.0
363504,C0-nP8lPWt0,"Covid tembus 40%,\nseperti dilansir oleh detik...",2023-12-18,id,Indonesian,neutral,2023-12-18,2.284120e+09,1.055558e+09,244519251.0,102.0
363505,C0_65IFupdt,My Companion Cleaning Services\nA clean enviro...,2023-12-18,en,English,positive,2023-12-18,2.284120e+09,1.055558e+09,244519251.0,102.0
363506,C0_cIDormR9,Last week to get your home cleaned and ready t...,2023-12-18,en,English,positive,2023-12-18,2.284120e+09,1.055558e+09,244519251.0,102.0
